# Loading Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from copy import deepcopy
import time, datetime
import pickle 

import IPython
import IPython.display
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import timeseries_dataset_from_array

import warnings
warnings.filterwarnings('ignore')

# Preprocessing data for building the ML model

### Loading the data

In [2]:
og_train_df = pd.read_csv('og_data/train.csv')
og_test_df = pd.read_csv('og_data/test.csv')
og_oil_df = pd.read_csv('og_data/oil.csv')
og_stores_df = pd.read_csv('og_data/stores.csv')
og_transactions_df = pd.read_csv('og_data/transactions.csv')
og_holidays_df = pd.read_csv('og_data/holidays_events.csv')

In [3]:
train_df = deepcopy(og_train_df)
test_df = deepcopy(og_test_df)
oil_df = deepcopy(og_oil_df)
stores_df = deepcopy(og_stores_df)
transactions_df = deepcopy(og_transactions_df) 

In [4]:
date_min, date_max = train_df.date.min(), test_df.date.max()

### Summary of original data

In [4]:
og_train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
og_oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [6]:
og_stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [7]:
og_transactions_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


### Remove the nan values from oil prices

In [5]:
new_df = pd.DataFrame()
new_df['date'] = [d.strftime('%Y-%m-%d') for d in pd.date_range(date_min, date_max)]
new_df = pd.merge(new_df,oil_df, how="outer", on='date')
new_df['dcoilwtico'] = new_df['dcoilwtico'].interpolate(method = "linear")
oil_df = new_df

In [6]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

In [7]:
oil_prices = np.asarray(oil_df['dcoilwtico'])

In [8]:
nans, x= nan_helper(oil_prices)
oil_prices[nans]= np.interp(x(nans), x(~nans), oil_prices[~nans])

In [9]:
oil_df['dcoilwtico'] = oil_prices

### Building the date conversion into unix time for ML model

In [10]:
date_min = time.mktime(datetime.datetime.strptime(date_min, "%Y-%m-%d").timetuple())
date_max = time.mktime(datetime.datetime.strptime(date_max, "%Y-%m-%d").timetuple())

In [11]:
og_test_df 

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [11]:
def GiveUnixTimeListNormalized(vector, date_format="%Y-%m-%d"):
    new_dates = np.zeros(len(vector), dtype=float)
    for i, s in enumerate(vector):
        new_dates[i] = time.mktime(datetime.datetime.strptime(s, date_format).timetuple())
    new_dates -= date_min
    new_dates /= (date_max - date_min)
    return new_dates

In [12]:
new_dates = GiveUnixTimeListNormalized(oil_df['date'], date_format="%Y-%m-%d")
oil_df['date'] = new_dates

In [13]:
new_dates = GiveUnixTimeListNormalized(og_train_df['date'], date_format="%Y-%m-%d")
train_df['date'] = new_dates

In [14]:
new_dates = GiveUnixTimeListNormalized(og_test_df['date'], date_format="%Y-%m-%d")
test_df['date'] = new_dates

In [16]:
train_df

,id,date,store_nbr,family,sales,onpromotion
0,0,0.000000,1,AUTOMOTIVE,0.000,0
1,1,0.000000,1,BABY CARE,0.000,0
2,2,0.000000,1,BEAUTY,0.000,0
3,3,0.000000,1,BEVERAGES,0.000,0
4,4,0.000000,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,0.990605,9,POULTRY,438.133,0
3000884,3000884,0.990605,9,PREPARED FOODS,154.553,1
3000885,3000885,0.990605,9,PRODUCE,2419.729,148
3000886,3000886,0.990605,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [17]:
oil_df

,date,dcoilwtico
0,0.000000,93.140000
1,0.000587,93.140000
2,0.001174,92.970000
3,0.001762,93.120000
4,0.002349,93.146667
...,...,...
1699,0.997651,46.816667
1700,0.998238,46.400000
1701,0.998826,46.460000
1702,0.999413,45.960000


In [18]:
test_df

,id,date,store_nbr,family,onpromotion
0,3000888,0.991192,1,AUTOMOTIVE,0
1,3000889,0.991192,1,BABY CARE,0
2,3000890,0.991192,1,BEAUTY,2
3,3000891,0.991192,1,BEVERAGES,20
4,3000892,0.991192,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,1.000000,9,POULTRY,1
28508,3029396,1.000000,9,PREPARED FOODS,0
28509,3029397,1.000000,9,PRODUCE,1
28510,3029398,1.000000,9,SCHOOL AND OFFICE SUPPLIES,9


### Building the processed dictionaries to train and test the ML model

In [30]:
stores_transactions_dict = {}
for store in train_df['store_nbr'].unique():
    print(store)
    stores_transactions_dict[store] = {}
    first_cond = False
    for fam in train_df['family'].unique():
        if first_cond == False:
            stores_transactions_dict[store]['date'] = train_df.where((train_df['store_nbr'] == store) & (train_df['family'] == fam))['date'].dropna().values.tolist()
            oil_list = []
            for date in stores_transactions_dict[store]['date']:
                if date in oil_df['date'].tolist():
                    oil_list.append(oil_df.iloc[oil_df['date'].tolist().index(date)]['dcoilwtico'])
            stores_transactions_dict[store]['oil_prices'] = oil_list
            first_cond = True
            
        stores_transactions_dict[store]['sales_'+fam] = train_df.where((train_df['store_nbr'] == store) & (train_df['family'] == fam))['sales'].dropna().values.tolist()
        stores_transactions_dict[store]['onpromotion_'+fam] = train_df.where((train_df['store_nbr'] == store) & (train_df['family'] == fam))['onpromotion'].dropna().values.tolist()

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
41
42
43
44
45
46
47
48
49
5
50
51
52
53
54
6
7
8
9


In [31]:
with open('Data/train_dictionary.pkl', 'wb') as f:
    pickle.dump(stores_transactions_dict, f)

In [24]:
stores_transactions_test_dict = {}
for store in test_df['store_nbr'].unique():
    print(store)
    stores_transactions_test_dict[store] = {}
    first_cond = False
    for fam in test_df['family'].unique():
        if first_cond == False:
            stores_transactions_test_dict[store]['date'] = test_df.where((test_df['store_nbr'] == store) & (test_df['family'] == fam))['date'].dropna().values.tolist()
            oil_list = []
            for date in stores_transactions_test_dict[store]['date']:
                if date in oil_df['date'].tolist():
                    oil_list.append(oil_df.iloc[oil_df['date'].tolist().index(date)]['dcoilwtico'])
            stores_transactions_test_dict[store]['oil_prices'] = oil_list
            first_cond=True

        stores_transactions_test_dict[store]['onpromotion_'+fam] = test_df.where((test_df['store_nbr'] == store) & (test_df['family'] == fam))['onpromotion'].dropna().values.tolist()

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
41
42
43
44
45
46
47
48
49
5
50
51
52
53
54
6
7
8
9


In [29]:
with open('Data/test_dictionary.pkl', 'wb') as f:
    pickle.dump(stores_transactions_test_dict, f)

# From here we work with the dictionary processed data

In [2]:
og_test_df = pd.read_csv('og_data/test.csv')

In [28]:
stores_transactions_test_dict[1].keys()

dict_keys(['date', 'oil_prices', 'onpromotion_AUTOMOTIVE', 'onpromotion_BABY CARE', 'onpromotion_BEAUTY', 'onpromotion_BEVERAGES', 'onpromotion_BOOKS', 'onpromotion_BREAD/BAKERY', 'onpromotion_CELEBRATION', 'onpromotion_CLEANING', 'onpromotion_DAIRY', 'onpromotion_DELI', 'onpromotion_EGGS', 'onpromotion_FROZEN FOODS', 'onpromotion_GROCERY I', 'onpromotion_GROCERY II', 'onpromotion_HARDWARE', 'onpromotion_HOME AND KITCHEN I', 'onpromotion_HOME AND KITCHEN II', 'onpromotion_HOME APPLIANCES', 'onpromotion_HOME CARE', 'onpromotion_LADIESWEAR', 'onpromotion_LAWN AND GARDEN', 'onpromotion_LINGERIE', 'onpromotion_LIQUOR,WINE,BEER', 'onpromotion_MAGAZINES', 'onpromotion_MEATS', 'onpromotion_PERSONAL CARE', 'onpromotion_PET SUPPLIES', 'onpromotion_PLAYERS AND ELECTRONICS', 'onpromotion_POULTRY', 'onpromotion_PREPARED FOODS', 'onpromotion_PRODUCE', 'onpromotion_SCHOOL AND OFFICE SUPPLIES', 'onpromotion_SEAFOOD'])

In [3]:
with open('Data/train_dictionary.pkl', 'rb') as f:
    stores_transactions_dict = pickle.load(f)

with open('Data/test_dictionary.pkl', 'rb') as f:
    stores_transactions_test_dict = pickle.load(f)

In [49]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
    self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([f'Total window size: {self.total_window_size}', f'Input indices: {self.input_indices}', f'Label indices: {self.label_indices}', 
                      f'Label column name(s): {self.label_columns}'])
  
def split_window(self, features):
  inputs = features[:, self.input_slice, :] 
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

def plot(self, model=None, plot_col='sales_AUTOMOTIVE', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time (days)')

WindowGenerator.plot = plot

def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    return tf.tile(inputs[:, -1:, :], [1, OUT_STEPS, 1])
  
class RepeatBaseline(tf.keras.Model):
  def call(self, inputs):
    return inputs
  
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)

def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the LSTM state.
  prediction, state = self.warmup(inputs)

  # Insert the first prediction.
  predictions.append(prediction)

  # Run the rest of the prediction steps.
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output.
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  return predictions

FeedBack.call = call

In [68]:
MAX_EPOCHS = 100

def compile_and_fit(model, window, patience=30):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-7, patience=patience, restore_best_weights=True)

  model.compile(loss=tf.keras.losses.MeanSquaredLogarithmicError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [36]:
df = pd.DataFrame.from_dict(stores_transactions_dict[key])
test_df = pd.DataFrame.from_dict(stores_transactions_test_dict[key])

In [40]:
df_features = df.filter(test_df.columns, axis=1)
df_timesteps = df.drop(test_df.columns, axis=1)

In [41]:
df_timesteps

,sales_AUTOMOTIVE,sales_BABY CARE,sales_BEAUTY,sales_BEVERAGES,sales_BOOKS,sales_BREAD/BAKERY,sales_CELEBRATION,sales_CLEANING,sales_DAIRY,sales_DELI,...,sales_MAGAZINES,sales_MEATS,sales_PERSONAL CARE,sales_PET SUPPLIES,sales_PLAYERS AND ELECTRONICS,sales_POULTRY,sales_PREPARED FOODS,sales_PRODUCE,sales_SCHOOL AND OFFICE SUPPLIES,sales_SEAFOOD
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.0,0.000,0.0,0.0,0.0,0.000,0.00000,0.000,0.0,0.000000
1,5.0,0.0,6.0,1309.0,0.0,1001.0,0.0,2624.0,473.0,725.902,...,0.0,374.531,482.0,0.0,0.0,651.292,83.00000,0.000,0.0,29.214000
2,22.0,0.0,6.0,1047.0,0.0,747.0,0.0,1781.0,231.0,414.087,...,0.0,400.863,372.0,0.0,0.0,509.496,66.00000,0.000,0.0,25.000000
3,0.0,0.0,3.0,986.0,0.0,513.0,0.0,1383.0,205.0,238.125,...,0.0,310.878,324.0,0.0,0.0,332.672,57.00000,0.000,0.0,11.000000
4,27.0,0.0,9.0,1415.0,0.0,631.0,0.0,2259.0,416.0,472.871,...,0.0,447.685,461.0,0.0,0.0,510.919,84.00000,0.000,0.0,28.361000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,18.0,1.0,4.0,2619.0,0.0,387.0,10.0,1496.0,738.0,636.690,...,5.0,309.245,373.0,11.0,2.0,525.224,112.10000,1453.078,140.0,23.831000
1680,8.0,0.0,14.0,3374.0,0.0,457.0,18.0,1341.0,767.0,477.783,...,2.0,260.298,400.0,7.0,10.0,383.387,129.90399,1419.264,138.0,16.859001
1681,15.0,0.0,15.0,3752.0,0.0,485.0,11.0,1655.0,795.0,503.209,...,3.0,327.206,510.0,2.0,9.0,412.458,105.16900,1693.607,200.0,20.000000
1682,8.0,0.0,4.0,3367.0,0.0,447.0,12.0,1429.0,746.0,493.971,...,12.0,330.975,445.0,2.0,14.0,283.429,114.12000,1348.425,182.0,17.000000


In [42]:
df_features

,date,oil_prices,onpromotion_AUTOMOTIVE,onpromotion_BABY CARE,onpromotion_BEAUTY,onpromotion_BEVERAGES,onpromotion_BOOKS,onpromotion_BREAD/BAKERY,onpromotion_CELEBRATION,onpromotion_CLEANING,...,onpromotion_MAGAZINES,onpromotion_MEATS,onpromotion_PERSONAL CARE,onpromotion_PET SUPPLIES,onpromotion_PLAYERS AND ELECTRONICS,onpromotion_POULTRY,onpromotion_PREPARED FOODS,onpromotion_PRODUCE,onpromotion_SCHOOL AND OFFICE SUPPLIES,onpromotion_SEAFOOD
0,0.000000,93.140000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000587,93.140000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.001174,92.970000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.001762,93.120000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.002349,93.146667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,0.988256,48.810000,0.0,0.0,1.0,25.0,0.0,7.0,0.0,24.0,...,0.0,0.0,11.0,0.0,0.0,22.0,3.0,6.0,7.0,0.0
1680,0.988843,48.403333,0.0,0.0,1.0,27.0,0.0,4.0,0.0,25.0,...,0.0,0.0,7.0,0.0,0.0,0.0,1.0,7.0,10.0,4.0
1681,0.989431,47.996667,0.0,0.0,1.0,31.0,0.0,70.0,0.0,22.0,...,0.0,0.0,9.0,0.0,0.0,0.0,1.0,7.0,8.0,0.0
1682,0.990018,47.590000,0.0,0.0,1.0,28.0,0.0,6.0,0.0,23.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,7.0,11.0,0.0


In [77]:
lstm_model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_47 (LSTM)              (None, 16, 256)           296960    
                                                                 
 batch_normalization_45 (Bat  (None, 16, 256)          1024      
 chNormalization)                                                
                                                                 
 dropout_43 (Dropout)        (None, 16, 256)           0         
                                                                 
 lstm_48 (LSTM)              (None, 16, 256)           525312    
                                                                 
 batch_normalization_46 (Bat  (None, 16, 256)          1024      
 chNormalization)                                                
                                                                 
 dropout_44 (Dropout)        (None, 16, 256)         

In [78]:
for key in stores_transactions_dict.keys():
    if key == 1:
        df = pd.DataFrame.from_dict(stores_transactions_dict[key])
        test_df = pd.DataFrame.from_dict(stores_transactions_test_dict[key])

        df_features = df.filter(test_df.columns, axis=1)
        df_timesteps = df.drop(test_df.columns, axis=1)

        column_indices = {name: i for i, name in enumerate(df.columns)}

        n = len(df_timesteps)
        num_features = df_timesteps.shape[1]
        OUT_STEPS =  pd.DataFrame.from_dict(stores_transactions_test_dict[key]).shape[0] #test_df.shape[1]
        #print(n, num_features)
        train_df = df_timesteps[0:int(n*0.7)]
        val_df = df_timesteps[int(n*0.7):int(n*0.9)]
        test_df = df_timesteps[int(n*0.9):] #pd.DataFrame.from_dict(stores_transactions_test_dict[key])

        train_mean = train_df.mean()
        train_std = train_df.std()
        train_std = train_std.replace(0, 1.0)

        train_df = (train_df - train_mean) / train_std
        val_df = (val_df - train_mean) / train_std
        test_df = (test_df - train_mean) / train_std

        plot = False
        if plot:
            df_std = (df - train_mean) / train_std
            df_std = df_std.melt(var_name='Column', value_name='Normalized')
            plt.figure(figsize=(30, 20))
            ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
            _ = ax.set_xticklabels(df.keys(), rotation=90)

        label_columns = train_df.columns #['sales_AUTOMOTIVE']        
        wide_window = WindowGenerator(input_width=OUT_STEPS, label_width=OUT_STEPS, shift=1, label_columns=label_columns)

        train_single = True
        if train_single:
            val_performance = { }
            performance = {}

            lstm_model = tf.keras.models.Sequential([
                # Shape [batch, time, features] => [batch, time, lstm_units]
                tf.keras.layers.LSTM(256, return_sequences=True),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.LSTM(256, return_sequences=True),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.LSTM(256, return_sequences=True),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.Dense(512),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.Dense(256),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.2),
                # Shape => [batch, time, features]
                tf.keras.layers.Dense(units=num_features)
            ])

            history = compile_and_fit(lstm_model, wide_window)

            IPython.display.clear_output()
            val_performance['LSTM'] = lstm_model.evaluate(wide_window.val, return_dict=True)
            performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0, return_dict=True)

            wide_window.plot(lstm_model, label_columns=['sales_AUTOMOTIVE']  )


        input_width=30
        multi_window = WindowGenerator(input_width=input_width, label_width=OUT_STEPS, shift=OUT_STEPS, label_columns=['sales_AUTOMOTIVE'])
        multi_val_performance = {}
        multi_performance = {}
        
        train_dense = False
        if train_dense:

            multi_dense_model = tf.keras.Sequential([
                # Take the last time step.
                # Shape [batch, time, features] => [batch, 1, features]
                tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
                # Shape => [batch, 1, dense_units]
                tf.keras.layers.LSTM(units=200, return_sequences=True),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.LSTM(units=200, return_sequences=True),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.Dense(256, activation='relu'),
                
                # Shape => [batch, out_steps*features]
                tf.keras.layers.Dense(OUT_STEPS*num_features,
                                    kernel_initializer=tf.initializers.zeros()),
                # Shape => [batch, out_steps, features]
                tf.keras.layers.Reshape([OUT_STEPS, num_features])
            ])

            history = compile_and_fit(multi_dense_model, multi_window)

            IPython.display.clear_output()
            multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val, return_dict=True)
            multi_performance['Dense'] = multi_dense_model.evaluate(multi_window.test, verbose=0, return_dict=True)
            multi_window.plot(multi_dense_model)

        train_conv = False
        if train_conv:
            CONV_WIDTH = 3
            multi_conv_model = tf.keras.Sequential([
                # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
                tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
                # Shape => [batch, 1, conv_units]
                tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
                # Shape => [batch, 1,  out_steps*features]
                tf.keras.layers.Dense(OUT_STEPS*num_features,
                                    kernel_initializer=tf.initializers.zeros()),
                # Shape => [batch, out_steps, features]
                tf.keras.layers.Reshape([OUT_STEPS, num_features])
            ])

            history = compile_and_fit(multi_conv_model, multi_window)

            IPython.display.clear_output()

            multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val, return_dict=True)
            multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0, return_dict=True)
            multi_window.plot(multi_conv_model)

        train_feedback = False
        if train_feedback:
            
            feedback_model = FeedBack(units=256, out_steps=OUT_STEPS)

            history = compile_and_fit(feedback_model, multi_window)

            #IPython.display.clear_output()

            multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val, return_dict=True)
            multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test, verbose=0, return_dict=True)
            multi_window.plot(feedback_model)
        

11/11 [==============================] - 1s 29ms/step - loss: 0.1248 - mean_absolute_error: 0.6494


TypeError: plot() got an unexpected keyword argument 'label_columns'

In [85]:
multi_window

Total window size: 46
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
Label indices: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45]
Label column name(s): ['sales_AUTOMOTIVE']

In [92]:
test_df

,sales_AUTOMOTIVE,sales_BABY CARE,sales_BEAUTY,sales_BEVERAGES,sales_BOOKS,sales_BREAD/BAKERY,sales_CELEBRATION,sales_CLEANING,sales_DAIRY,sales_DELI,...,sales_MAGAZINES,sales_MEATS,sales_PERSONAL CARE,sales_PET SUPPLIES,sales_PLAYERS AND ELECTRONICS,sales_POULTRY,sales_PREPARED FOODS,sales_PRODUCE,sales_SCHOOL AND OFFICE SUPPLIES,sales_SEAFOOD
1515,-0.331311,0.0,-0.047837,-0.463858,0.0,-1.755420,-0.015378,-1.487896,-1.095015,-1.412901,...,0.748220,-1.252290,-1.318772,-0.417731,0.278431,-0.519535,-1.173890,0.331445,-0.290013,-1.094658
1516,-0.712820,0.0,0.558103,3.021237,0.0,2.307346,0.163977,1.840182,2.877839,1.122037,...,2.978812,0.315032,1.565772,1.760852,1.577775,1.041035,1.284529,3.001934,-0.290013,1.642333
1517,0.050199,0.0,3.587803,1.763917,0.0,0.851203,0.368955,1.354837,1.320440,0.228761,...,2.978812,0.388527,1.868184,-0.145408,0.464051,0.351572,1.579196,1.385273,1.987554,0.275923
1518,0.813217,0.0,4.193743,1.789857,0.0,1.404789,0.368955,1.257768,1.528755,1.518669,...,0.748220,1.514766,1.123786,2.850143,0.464051,2.155613,1.548459,1.141831,-0.290013,2.122862
1519,1.194726,0.0,1.769983,1.852418,1.0,1.253864,-0.015378,0.559796,1.568434,0.309752,...,1.066876,-0.362072,0.286337,1.488529,1.020913,-0.163586,0.597106,1.237690,-0.290013,0.782221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,-0.712820,0.0,-0.653777,-0.544729,0.0,-1.648142,-0.092244,-1.339981,-1.184292,-1.355081,...,0.748220,0.431095,-1.574659,0.671560,-0.092810,-0.125031,-1.868004,0.068289,-0.290013,-0.570378
1680,1.194726,0.0,0.558103,0.451666,0.0,-0.726745,-0.117866,-1.293758,-0.276637,-0.495940,...,1.385532,-0.639343,2.473008,0.126914,0.464051,-0.423883,-1.083615,0.570324,-0.290013,-0.503082
1681,-0.712820,0.0,-0.653777,-0.854481,0.0,-1.732416,-0.169111,-2.135022,-1.566202,-1.758745,...,0.748220,-1.398524,-1.760758,-0.145408,-0.649672,-1.158548,-2.024814,-0.031523,-0.290013,-1.316198
1682,-0.712820,0.0,2.375923,1.278690,0.0,0.243784,-0.092244,-0.276845,0.581420,0.849200,...,2.022844,-0.756027,0.891161,0.126914,0.464051,-0.046241,-0.270324,1.229140,-0.290013,-1.061196


In [109]:
multi_window.test

<MapDataset element_spec=(TensorSpec(shape=(None, 30, 33), dtype=tf.float32, name=None), TensorSpec(shape=(None, 16, 1), dtype=tf.float32, name=None))>

In [95]:
prediction = lstm_model.predict(multi_window.test)

4/4 [==============================] - 0s 65ms/step


In [100]:
predict_df = pd.DataFrame(prediction[0], columns=test_df.columns)

In [102]:
predict_df = (predict_df * train_std) + train_mean

In [105]:
predict_df.round()

,sales_AUTOMOTIVE,sales_BABY CARE,sales_BEAUTY,sales_BEVERAGES,sales_BOOKS,sales_BREAD/BAKERY,sales_CELEBRATION,sales_CLEANING,sales_DAIRY,sales_DELI,...,sales_MAGAZINES,sales_MEATS,sales_PERSONAL CARE,sales_PET SUPPLIES,sales_PLAYERS AND ELECTRONICS,sales_POULTRY,sales_PREPARED FOODS,sales_PRODUCE,sales_SCHOOL AND OFFICE SUPPLIES,sales_SEAFOOD
0,4.0,-0.0,3.0,2085.0,-0.0,408.0,17.0,762.0,760.0,148.0,...,5.0,339.0,153.0,5.0,9.0,337.0,98.0,2541.0,0.0,31.0
1,4.0,-0.0,2.0,2100.0,-0.0,402.0,18.0,765.0,744.0,147.0,...,5.0,322.0,149.0,5.0,9.0,307.0,98.0,2511.0,0.0,29.0
2,4.0,-0.0,3.0,1969.0,-0.0,374.0,18.0,695.0,673.0,137.0,...,6.0,328.0,140.0,5.0,8.0,281.0,94.0,2106.0,0.0,27.0
3,5.0,-0.0,3.0,2125.0,-0.0,393.0,20.0,714.0,726.0,156.0,...,6.0,563.0,137.0,5.0,8.0,412.0,103.0,2184.0,0.0,37.0
4,4.0,-0.0,3.0,2147.0,-0.0,379.0,12.0,639.0,756.0,137.0,...,6.0,277.0,155.0,6.0,8.0,274.0,96.0,2254.0,0.0,28.0
5,3.0,-0.0,2.0,1368.0,-0.0,307.0,4.0,535.0,549.0,105.0,...,4.0,209.0,110.0,4.0,5.0,201.0,76.0,1211.0,0.0,21.0
6,4.0,-0.0,3.0,2032.0,-0.0,402.0,15.0,688.0,742.0,144.0,...,6.0,355.0,146.0,5.0,8.0,339.0,100.0,2305.0,0.0,30.0
7,4.0,-0.0,3.0,1801.0,-0.0,370.0,12.0,647.0,681.0,129.0,...,4.0,255.0,138.0,5.0,7.0,262.0,93.0,1965.0,0.0,26.0
8,4.0,-0.0,3.0,1916.0,-0.0,388.0,14.0,680.0,715.0,136.0,...,5.0,316.0,141.0,5.0,8.0,307.0,99.0,2220.0,0.0,29.0
9,4.0,-0.0,2.0,1787.0,-0.0,363.0,13.0,652.0,646.0,126.0,...,5.0,280.0,134.0,4.0,8.0,261.0,90.0,1910.0,0.0,26.0
